In [ ]:
import os
import time
import datetime as dt
import glob
from multiprocessing import Pool

import tqdm
import numpy as np
from scipy.interpolate import RectBivariateSpline

In [ ]:
ar_lim = [0.97642671, 1.02437728]
force = False
lag_dir = '/data2/Anton/sia/cdr_1991_2023'
years = range(2020,2025)

#lag_dir = '/data2/Anton/aria_sia'
#years = range(1970,2040)

In [ ]:
def find_minimum(year):
    start_dates = [dt.datetime(year,9,5) + dt.timedelta(i) for i in range(11)]
    stop_date = dt.datetime(year,9,15)
    sic_dst_file = stop_date.strftime(f'{lag_dir}/sic/%Y/sic_%Y%m%d_%Y%m%d.npz')
    if os.path.exists(sic_dst_file) and not force:
        print(f'{sic_dst_file} exists')
        return
    c1s = []
    for start_i, start_date in enumerate(start_dates):
        sic_file = start_date.strftime(f'{lag_dir}/sic/%Y/sic_%Y%m%d.npz')
        c0 = np.load(sic_file)['c']
        prop_dates = start_dates[start_i:]
        for prop_date in prop_dates[1:]:
            # propagate SIC
            mesh_file = prop_date.strftime(f'{lag_dir}/mesh/%Y/mesh_%Y%m%d.npz')
            src2dst = np.load(mesh_file)['src2dst']
            weights = np.load(mesh_file)['weights']
            ar = np.load(mesh_file)['ar']
            #ar = np.clip(ar, *ar_lim)
            c1 = np.zeros(src2dst[:,1].max()+1)
            np.add.at(c1, src2dst[:,1], c0[src2dst[:,0]] * weights)
            c1 /= ar
            c1 = np.clip(c1, 0, 100)

            # cap by observed SIC
            sic_file = prop_date.strftime(f'{lag_dir}/sic/%Y/sic_%Y%m%d.npz')
            c1_obs = np.load(sic_file)['c']
            c1[c1 > c1_obs] = c1_obs[c1 > c1_obs]
            c0 = c1
        c1s.append(c1)

    c1_min = np.array(c1s).min(axis=0)
    np.savez(sic_dst_file, c=c1_min)
    print(sic_dst_file)

for year in years:
    find_minimum(year)
